### Vorbereitung
Diese Dinge brauche ich:
- camelot zum Verarbeiten des PDF
- numpy zum Einsetzen von NaN

In [12]:
import pandas as pd
import camelot
import numpy as np

### File konvertieren
- der flavour "stream", der mit Textabständen operiert, funktionierte besser
- ich verarbeite gleich sämtliche Seiten auf einmal (dauert etwas)

In [13]:
pfad = "Lieferung_SEB-Schulergebnisse_2/Schulberichte/AlleSchulen_2006.pdf"
umfrage = camelot.read_pdf(pfad, flavor='stream', pages='all')

### Datenverarbeitung
Zunächst definiere ich hier zwei Dinge, die später brauche
- das Jahr (manuell, um es für andere Jahrgänge leichter anzupassen)
- die dic_list, in der meine Daten zusammenfliessen sollen

In [15]:
jahr = 2006
dic_list = []

Jetzt kommt **der Loop**, der im Wesentlichen folgendes mache:
- ich öffne jede zweite Tabelle meines kovertierten Files (weil jedes zweite leer ist)
- weil die Tabellen fiese kleine Abweichungen haben, überprüfe ich, ob es Anpassungen in der Höhe und der Breite braucht
- nach den allfälligen Anpassungen berechne ich die gewünschten Notenschitte
- diese Daten ziehe ich dann raus und fürge sie (mit Jahr und Schulname) in ein df, das ich als csv speichere
Die Details sind im Folgenden im Code vermerkt:

In [16]:
for table in range(0, len(umfrage), 2):
    df_schule = umfrage[table].df
    
    if df_schule.loc[0,13] == "Beurteilung":   #Wir checken hier anhand eines eindeutigen Merkmals, ob eine Zeile fehlt
        df_newrow = df_schule.head(1)
        df_schule = pd.concat([df_newrow, df_schule], ignore_index = True)   # Ignore Index ist entscheidend!

    name = df_schule.loc[2,0]
        
    if df_schule.loc[1,12] == "Beurteilung": # wiederholen den CHeck wegen Spaltenverschiebungen
        df_kurz = df_schule.drop([0,1,2,3,4,5,81,82])  # auch Schluss kürzen, weils da Müll drinhat, der stört
        df_kurz = df_kurz.replace("", np.nan)
        df_kurz.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] # Kolumnenamen umschreiben
    
    else:
        df_kurz = df_schule.drop([0,1,2,3,4,5])
        df_kurz = df_kurz.replace("", np.nan)
    
    our_columns = [3,8,9,10,11,12,13]
    df_kurz[our_columns] = df_kurz[our_columns].astype(float)
    df_kurz["Summe"] = df_kurz[8] + df_kurz[9].multiply(2) + df_kurz[10].multiply(3) + df_kurz[11].multiply(4) + df_kurz[12].multiply(5) + df_kurz[13].multiply(6)
    df_kurz["Notenschnitt"] = (df_kurz["Summe"] / df_kurz[3]).round(3) #gerundet auf eine Kommastelle
    
    zufr = df_kurz.loc[6, "Notenschnitt"]
    vorb_f = df_kurz.loc[7, "Notenschnitt"]
    vorb_p = df_kurz.loc[9, "Notenschnitt"]
    prob = df_kurz.loc[13, "Notenschnitt"]
    urt = df_kurz.loc[14, "Notenschnitt"]
    zus = df_kurz.loc[16, "Notenschnitt"]
    konf = df_kurz.loc[17, "Notenschnitt"]

    mini_dict = {"jahr": jahr, 
                 "name": name, 
                 "zufr": zufr, 
                 "vorb_f": vorb_f, 
                 "vorb_p":vorb_p, 
                 "prob": prob, 
                 "urt": urt, 
                 "zus": zus, 
                 "konf": konf}
    dic_list.append(mini_dict)

In [17]:
df_final = pd.DataFrame(dic_list)
df_final.to_csv("Pandas_Results2006.csv")

In [18]:
df_final

,jahr,name,zufr,vorb_f,vorb_p,prob,urt,zus,konf
0,2006,Büelrain,4.815,4.177,4.065,4.277,4.200,4.154,3.738
1,2006,Enge,4.746,4.074,3.926,4.493,4.437,3.986,3.761
2,2006,Freudenberg,4.316,4.000,3.868,4.526,4.658,4.079,3.842
3,2006,Glattal,4.833,3.941,4.118,4.500,4.333,3.722,3.444
4,2006,Hohe Promenade,5.091,4.758,4.636,4.706,4.706,4.000,4.000
5,2006,Hottingen,4.051,3.500,3.711,3.974,4.000,3.821,3.513
6,2006,Im Lee,4.676,4.406,3.971,4.603,4.473,3.836,3.493
7,2006,KME,5.037,4.173,4.257,4.309,4.185,3.963,3.407
8,2006,Küsnacht,5.095,4.119,4.524,4.738,5.095,4.000,4.048
9,2006,Liceo Artistico,5.450,4.579,4.895,4.950,4.950,4.950,4.263
